In [22]:
# === Power Generation Calculation ===
import pandas as pd
import numpy as np

# === Step 1: Load MCDA data ===
mcda_path = r"C:\GISDataManipulation\MCDA Files\MCDAwEnvironment\mcda_with_environmental_filter.txt"
mcda = pd.read_csv(mcda_path)

# === Step 2: Filter for high-scoring locations only ===
mcda_filtered = mcda[mcda["final_score_with_env"] > 7.5].copy()
print(f"Filtered locations with final_score_with_env > 7.5: {len(mcda_filtered)}")

# === Step 3: Estimate Solar Energy ===
mcda_filtered["Installed_kWp"] = 50
mcda_filtered["Monthly_Energy_kWh"] = mcda_filtered["PVOUT_Average"] * mcda_filtered["Installed_kWp"]
mcda_filtered["Yearly_Energy_kWh_Solar"] = mcda_filtered["Monthly_Energy_kWh"] * 12

# === Step 4: Estimate Wind Energy for 5 turbines ===
def estimate_wind_energy(v, rotor_area=78.5, air_density=1.225, efficiency=0.35, hours=8760, turbines=5):
    if np.isnan(v):
        return 0
    power = 0.5 * air_density * rotor_area * (v ** 3) * efficiency  # in watts
    energy_kwh = power * hours / 1000  # convert to kWh
    return energy_kwh * turbines  # scale for number of turbines

mcda_filtered["Yearly_Energy_kWh_Wind"] = mcda_filtered["Wind_Speed"].apply(estimate_wind_energy)

# === Step 5: Total energy and convert to MWh ===
mcda_filtered["Total_Yearly_Energy_kWh"] = (
    mcda_filtered["Yearly_Energy_kWh_Solar"] + mcda_filtered["Yearly_Energy_kWh_Wind"]
)
mcda_filtered["Total_Yearly_Energy_MWh"] = mcda_filtered["Total_Yearly_Energy_kWh"] / 1000

# === Step 6: Save selected columns only ===
output_path = r"C:\GISDataManipulation\MCDA Files\MCDAwEnvironment\filtered_energy_output.txt"
columns_to_save = [
    "Longitude",
    "Latitude",
    "PVOUT_Average",
    "Wind_Speed",
    "Total_Yearly_Energy_MWh"
]
mcda_filtered[columns_to_save].to_csv(output_path, index=False)

# === Step 7: Print summary ===
total_energy_mwh = mcda_filtered["Total_Yearly_Energy_MWh"].sum()
print("Final output contains only high-scoring locations (score > 7.5).")
print(f"Total yearly energy for filtered sites (with 5 turbines each): {total_energy_mwh:,.2f} MWh")
print(f"Saved to: {output_path}")


Filtered locations with final_score_with_env > 7.5: 15
Final output contains only high-scoring locations (score > 7.5).
Total yearly energy for filtered sites (with 5 turbines each): 8,327.63 MWh
Saved to: C:\GISDataManipulation\MCDA Files\MCDAwEnvironment\filtered_energy_output.txt


In [23]:
# === Map of Power Generation with Location Pin ===
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# === Step 1: Load the filtered energy output file ===
data_path = r"C:\GISDataManipulation\MCDA Files\MCDAwEnvironment\filtered_energy_output.txt"
df = pd.read_csv(data_path)

# === Step 2: Create the Folium map ===
m = folium.Map(location=[df["Latitude"].mean(), df["Longitude"].mean()], zoom_start=6, tiles="CartoDB positron")
marker_cluster = MarkerCluster().add_to(m)

# === Step 3: Add markers with full data popups ===
for _, row in df.iterrows():
    popup_content = (
        f"<b>Latitude:</b> {row['Latitude']}<br>"
        f"<b>Longitude:</b> {row['Longitude']}<br>"
        f"<b>PVOUT Average:</b> {row['PVOUT_Average']} kWh/kWp/Month<br>"
        f"<b>Wind Speed:</b> {row['Wind_Speed']} m/s<br>"
        f"<b>Total Yearly Energy:</b> {row['Total_Yearly_Energy_MWh']:.2f} MWh"
    )
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=folium.Popup(popup_content, max_width=300),
        icon=folium.Icon(color="blue", icon="bolt")
    ).add_to(marker_cluster)

# === Step 4: Save the map ===
map_output = r"C:\GISDataManipulation\MCDA Files\MCDAwEnvironment\filtered_energy_map.html"
m.save(map_output)
print(f"Map saved to: {map_output}")


Map saved to: C:\GISDataManipulation\MCDA Files\MCDAwEnvironment\filtered_energy_map.html
